In [49]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.cm as cm


In [50]:
V1='set_view (\
    -0.107686043,    0.598249078,   -0.794034779,\
     0.459203809,    0.738308251,    0.493982822,\
     0.881771803,   -0.311431885,   -0.354227066,\
     0.000000000,    0.000000000, -282.540802002,\
    18.762184143,   51.719478607,   62.844505310,\
   202.551422119,  362.530181885,  -20.000000000 )'
V2='set_view (\
     0.820026278,    0.477390289,   -0.315668732,\
    -0.477695227,    0.874692917,    0.081873596,\
     0.315201789,    0.083654091,    0.945322216,\
     0.000000000,    0.000000000, -320.854736328,\
    18.762184143,   51.719478607,   62.844505310,\
   240.865509033,  400.844116211,  -20.000000000 )'
V3='set_view (\
     0.387390643,    0.577329457,   -0.718753934,\
    -0.247406870,    0.816135585,    0.522201657,\
     0.888090014,   -0.024472144,    0.459000111,\
     0.000000000,    0.000000000, -280.427032471,\
    18.762184143,   51.719478607,   62.844505310,\
   200.437667847,  360.416412354,  -20.000000000 )'

In [51]:
def getIndexInChain(v,vc):
    v_=np.intersect1d(v,vc)
    v_=v_-vc[0]+1
    return v_

def pymolwrite(df,pymolfile='script.pml',CMAP='jet',
               PDBFILE='4edb.pdb',FILE='rndfile',DPI=300,
               chainspec=None,OUTPNG=True):
    '''
    ardict is in teh format: {key0:array,key1:array,...}
    where key is interpreted as an intensity
    '''
    
    MINVAL=0.01
    MAX=df.max().values[0]
    MIN=np.max([df.min().values[0],MINVAL])
    STEPS=10.0
    RNG=np.arange(MIN,MAX,(MAX-MIN)/STEPS)
    #norm = matplotlib.colors.LogNorm(vmin=MIN, vmax=MAX, clip=True)
    norm = matplotlib.colors.Normalize(vmin=MIN, vmax=MAX, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=CMAP)

    ardict={}
    for i in RNG[:-1]:
        ar=df[df.pr.between(i,i+STEPS)].sort_index().index.values
        if i>MINVAL:
            ardict[mapper.to_rgba(i)[:3]]=ar

    FRAG0='run ./color_L1.py'
    FRAG4='set ray_opaque_background, 0'
    FRAG5='show surface'

    FILE=CMAP+FILE
    
    f = open(CMAP+pymolfile, "w")
    f.write(FRAG0+'\n')
    f.write('color grey80\n')
    
    
    FRAG1='color_L1 '+PDBFILE+','
    
    count=0
    for (k,v) in ardict.items():
        for (kc,vc) in chainspec.items():
            count=count+1
            chain=kc
            v_=getIndexInChain(v,vc)
            STR='set_color COL'+str(count)+', ['+','.join(np.array(list(k)).astype(str))+']'
            COL='COL'+str(count)
            nodes=list(v_.astype(int).astype(str))
            nodes_=' '.join(nodes[:])
            f.write(STR+'\n')
            if count==1:
                FRAG2=', '+COL+', 0,'+'1'+',0,grey80'
            else:
                FRAG2=', '+COL+', 0,'+'0'+',0,grey80'

            f.write(FRAG1+nodes_+FRAG2+','+chain+'\n')
            
    f.write(FRAG4+'\n')
    
    if OUTPNG:
        f.write(V1+'\n')
        f.write('png '+FILE+str(1)+', dpi='+str(DPI)+', ray=1\n')

        f.write(V2+'\n')
        f.write('png '+FILE+str(2)+', dpi='+str(DPI)+', ray=1\n')

        f.write(V3+'\n')
        f.write('png '+FILE+str(3)+', dpi='+str(DPI)+', ray=1\n')


        f.write(FRAG5+'\n')

        f.write(V1+'\n')
        f.write('png '+FILE+str(4)+', dpi='+str(DPI)+', ray=1\n')

        f.write(V2+'\n')
        f.write('png '+FILE+str(5)+', dpi='+str(DPI)+', ray=1\n')

        f.write(V3+'\n')
        f.write('png '+FILE+str(6)+', dpi='+str(DPI)+', ray=1\n')
    
     
    f.write('quit\n')
    f.close()
    
    return

In [52]:
df=pd.read_csv('data.csv',sep='\t',index_col=0)

In [53]:
CHAINSPEC={}
CHAINSPEC['A']=np.arange(1,344)
CHAINSPEC['C']=np.arange(1,344)
CHAINSPEC['E']=np.arange(1,344)
CHAINSPEC['B']=np.arange(344,520)
CHAINSPEC['D']=np.arange(344,520)
CHAINSPEC['F']=np.arange(344,520)
#CHAINSPEC

In [54]:
pymolwrite(df,pymolfile='script.pml',FILE='rndfile.png',DPI=1200,chainspec=CHAINSPEC,CMAP='rainbow')
pymolwrite(df,pymolfile='script.pml',FILE='rndfile.png',DPI=1200,chainspec=CHAINSPEC,CMAP='jet')
pymolwrite(df,pymolfile='script.pml',FILE='rndfile.png',DPI=1200,chainspec=CHAINSPEC,CMAP='viridis')


In [55]:
! cat rainbowscript.pml

run ./color_L1.py
color grey80
set_color COL1, [0.303921568627451,0.30315267411304353,0.9881654720812594]
color_L1 4edb.pdb,3 4 5 98 99 100 101 102 112 113 114 115 116 142 143 144 145 146 156 158 159 160 161 162 163 174 175 176 177 178 179 180 181 182 183 184 198 199 200 201 202 203 204 205 206 207 208 209 218 219 220 221 222 228 229 230 240 241 266 267 268 269 270 271 298 299 300 301 336 337 338 339 340, COL1, 0,1,0,grey80,A
set_color COL2, [0.303921568627451,0.30315267411304353,0.9881654720812594]
color_L1 4edb.pdb,3 4 5 98 99 100 101 102 112 113 114 115 116 142 143 144 145 146 156 158 159 160 161 162 163 174 175 176 177 178 179 180 181 182 183 184 198 199 200 201 202 203 204 205 206 207 208 209 218 219 220 221 222 228 229 230 240 241 266 267 268 269 270 271 298 299 300 301 336 337 338 339 340, COL2, 0,0,0,grey80,C
set_color COL3, [0.303921568627451,0.30315267411304353,0.9881654720812594]
color_L1 4edb.pdb,3 4 5 98 99 100 101 102 112 113 114 115 116 142 143 144 145 146 156 158 159 16